# Задание:
Разобраться с моделькой перевода как она устроена
запустить для перевода с русского на английский (при желании можно взять другие пары языков) два варианта с вниманием и без внимания
оценить качество насколько корректно переводит (для теста отобрать примеры с увеличением длины текста) (так как оценка визуальная достаточно 20-ти примеров в тестовой выборке)

# Neural machine translation

In [5]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

## Download and prepare the dataset

We'll use a language dataset provided by http://www.manythings.org/anki/

In [12]:
!wget http://www.manythings.org/anki/rus-eng.zip

--2022-07-02 10:50:06--  http://www.manythings.org/anki/rus-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14819554 (14M) [application/zip]
Saving to: ‘rus-eng.zip’

rus-eng.zip         100%[===================>]  14.13M  17.2MB/s    in 0.8s    

2022-07-02 10:50:07 (17.2 MB/s) - ‘rus-eng.zip’ saved [14819554/14819554]



In [13]:
#!mkdir rus-eng
!unzip rus-eng.zip -d rus-eng/

Archive:  rus-eng.zip
  inflating: rus-eng/rus.txt         
  inflating: rus-eng/_about.txt      


In [14]:
!ls /content/rus-eng/ -lah

total 71M
drwxr-xr-x 2 root root 4.0K Jul  2 10:50 .
drwxr-xr-x 1 root root 4.0K Jul  2 10:50 ..
-rw-r--r-- 1 root root 1.5K May  2 01:29 _about.txt
-rw-r--r-- 1 root root  71M May  2 01:29 rus.txt


In [15]:
# Download the file
path_to_file = "/content/rus-eng/rus.txt"

In [16]:
def preprocess_sentence(w):
  w = w.lower().strip()

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Zа-яА-Я?.!,']+", " ", w)

  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [7]:
preprocess_sentence("I can't go.")

"<start> i can't go . <end>"

In [17]:
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENG, RUS]
def create_dataset(path, num_examples):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:2]]  for l in lines[:num_examples]]

  return zip(*word_pairs)

In [18]:
en, ru = create_dataset(path_to_file, None)
print(en[0])
print(ru[0])

<start> go . <end>
<start> марш ! <end>


In [19]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

In [20]:
def load_dataset(path, num_examples=None):
  # creating cleaned input, output pairs
  targ_lang, inp_lang = create_dataset(path, num_examples)

  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

### Limit the size of the dataset to experiment faster (optional)


In [12]:
len(en), len(ru)

(444587, 444587)

In [21]:
# Try experimenting with the size of that dataset
num_examples = 100000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [22]:
inp_lang

In [23]:
input_tensor[0]

array([   1, 5674,   24,    2,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0], dtype=int32)

In [24]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

80000 80000 20000 20000


In [25]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [26]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
4 ----> я
9555 ----> струсил
3 ----> .
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
4 ----> i
3293 ----> chickened
86 ----> out
3 ----> .
2 ----> <end>


### Create a tf.data dataset

In [27]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 300
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
#dataset = dataset.batch(BATCH_SIZE, drop_remainder==True)
dataset = dataset.batch(BATCH_SIZE)

In [28]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 15]), TensorShape([64, 11]))

In [29]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

    #self.bigru_0 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(self.enc_units, 
    #                                            return_sequences=True, return_state=True))
    
    self.bigru = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(self.enc_units, 
                                                return_sequences=False, return_state=True))


    #self.gru = tf.keras.layers.GRU(self.enc_units,
    #                               return_sequences=False,
    #                               return_state=True,
    #                               recurrent_initializer='glorot_uniform')

    

  def call(self, x, hidden):
    x = self.embedding(x)
    #state = self.bigru_0(x, initial_state = hidden)
    #state = self.bigru(state)

    state = self.bigru(x, initial_state = hidden)
    #output, state = self.gru(x, initial_state = hidden)
    return state

  def initialize_hidden_state(self):
    return [tf.zeros((self.batch_sz, self.enc_units)) for i in range(2)]
    #return tf.zeros((self.batch_sz, self.enc_units))

In [30]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
encoder_states = encoder.initialize_hidden_state()
#state_c = encoder.initialize_hidden_state()
#encoder_states = [state_h, state_c]

# sample_hidden = encoder.initialize_hidden_state()
sample_hidden = encoder(example_input_batch, encoder_states)

# print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden))

Encoder Hidden state shape: (batch size, units) [<tf.Tensor: shape=(64, 2048), dtype=float32, numpy=
array([[-2.0716032e-03,  1.3614398e-02,  5.3938883e-03, ...,
        -7.3685851e-03, -4.1158078e-03,  8.6220708e-03],
       [-2.0446840e-03,  1.3694559e-02,  5.3544156e-03, ...,
        -4.0086787e-03, -1.8176429e-03,  6.6663893e-03],
       [-2.0065152e-03,  1.3695389e-02,  5.3953752e-03, ...,
        -1.2506780e-02, -4.4213586e-05,  1.1518727e-03],
       ...,
       [-2.0737289e-03,  1.3652696e-02,  5.3830408e-03, ...,
        -1.0708647e-02,  1.6229097e-03,  7.8544365e-03],
       [-2.1510720e-03,  1.3698903e-02,  5.2846023e-03, ...,
        -8.0346745e-03, -2.8387839e-03,  4.8408457e-03],
       [-2.1159113e-03,  1.3731044e-02,  5.4212408e-03, ...,
        -2.0975992e-03,  7.5553562e-03,  5.8971317e-03]], dtype=float32)>, <tf.Tensor: shape=(64, 1024), dtype=float32, numpy=
array([[-0.0020716 ,  0.0136144 ,  0.00539389, ..., -0.00956457,
         0.01592911, -0.01610116],
       [-

In [31]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    
    self.fc = tf.keras.layers.Dense(vocab_size)

  def call(self, x, hidden):
    # enc_output shape == (batch_size, max_length, hidden_size)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x, initial_state=hidden)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state

In [32]:
sample_hidden[0]

<tf.Tensor: shape=(64, 2048), dtype=float32, numpy=
array([[-2.0716032e-03,  1.3614398e-02,  5.3938883e-03, ...,
        -7.3685851e-03, -4.1158078e-03,  8.6220708e-03],
       [-2.0446840e-03,  1.3694559e-02,  5.3544156e-03, ...,
        -4.0086787e-03, -1.8176429e-03,  6.6663893e-03],
       [-2.0065152e-03,  1.3695389e-02,  5.3953752e-03, ...,
        -1.2506780e-02, -4.4213586e-05,  1.1518727e-03],
       ...,
       [-2.0737289e-03,  1.3652696e-02,  5.3830408e-03, ...,
        -1.0708647e-02,  1.6229097e-03,  7.8544365e-03],
       [-2.1510720e-03,  1.3698903e-02,  5.2846023e-03, ...,
        -8.0346745e-03, -2.8387839e-03,  4.8408457e-03],
       [-2.1159113e-03,  1.3731044e-02,  5.4212408e-03, ...,
        -2.0975992e-03,  7.5553562e-03,  5.8971317e-03]], dtype=float32)>

In [25]:
sample_hidden[1]

<tf.Tensor: shape=(64, 1024), dtype=float32, numpy=
array([[-0.00651659,  0.0015186 ,  0.00477847, ...,  0.00076088,
        -0.00164839,  0.00754175],
       [-0.00691843,  0.0016113 ,  0.00455411, ...,  0.00080611,
        -0.00164456,  0.00777414],
       [-0.0069491 ,  0.00160151,  0.00460188, ...,  0.00071912,
        -0.00172119,  0.00768685],
       ...,
       [-0.00744378,  0.00173575,  0.00437781, ...,  0.00074614,
        -0.00163993,  0.00778439],
       [-0.00695033,  0.00159173,  0.00471702, ...,  0.0007499 ,
        -0.00156053,  0.00777683],
       [-0.00725776,  0.00176134,  0.0044791 , ...,  0.00074841,
        -0.00161041,  0.00776347]], dtype=float32)>

In [33]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

decoder_sample_x, decoder_sample_h = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden[1])



In [27]:
decoder_sample_x.shape

TensorShape([64, 7334])

In [28]:
decoder_sample_h.shape

TensorShape([64, 1024])

## Define the optimizer and the loss function

In [34]:
optimizer = tf.keras.optimizers.Adam()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

## Checkpoints (Object-based saving)

In [35]:
checkpoint_dir = './training_nmt_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [31]:
# <s> w1 w2 w3 w4 <end>
# 0    t

In [36]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_hidden = encoder(inp, enc_hidden)

    #dec_hidden = enc_hidden
    dec_hidden = enc_hidden[1]

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden = decoder(dec_input, dec_hidden)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [47]:
EPOCHS = 20

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()

  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 4.7150
Epoch 1 Batch 100 Loss 2.0174
Epoch 1 Batch 200 Loss 1.8183
Epoch 1 Batch 300 Loss 1.5324
Epoch 1 Batch 400 Loss 1.5406
Epoch 1 Batch 500 Loss 1.5446
Epoch 1 Batch 600 Loss 1.4341
Epoch 1 Batch 700 Loss 1.3243
Epoch 1 Batch 800 Loss 1.2832
Epoch 1 Batch 900 Loss 1.2192
Epoch 1 Batch 1000 Loss 1.1860
Epoch 1 Batch 1100 Loss 1.1526
Epoch 1 Batch 1200 Loss 1.1915
Epoch 1 Loss 1.4937
Time taken for 1 epoch 67.18843007087708 sec

Epoch 2 Batch 0 Loss 0.9780
Epoch 2 Batch 100 Loss 0.9951
Epoch 2 Batch 200 Loss 0.8783
Epoch 2 Batch 300 Loss 0.9091
Epoch 2 Batch 400 Loss 0.8119
Epoch 2 Batch 500 Loss 0.9260
Epoch 2 Batch 600 Loss 0.8656
Epoch 2 Batch 700 Loss 0.7707
Epoch 2 Batch 800 Loss 0.6817
Epoch 2 Batch 900 Loss 0.6837
Epoch 2 Batch 1000 Loss 0.5662
Epoch 2 Batch 1100 Loss 0.6232
Epoch 2 Batch 1200 Loss 0.6124
Epoch 2 Loss 0.7957
Time taken for 1 epoch 58.21935558319092 sec

Epoch 3 Batch 0 Loss 0.5294
Epoch 3 Batch 100 Loss 0.4274
Epoch 3 Batch 200 Loss 0.501

## Translate

* The evaluate function is similar to the training loop, except we don't use *teacher forcing* here. The input to the decoder at each time step is its previous predictions along with the hidden state and the encoder output.
* Stop predicting when the model predicts the *end token*.
* And store the *attention weights for every time step*.

Note: The encoder output is calculated only once for one input.

In [48]:
def evaluate(sentence):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = preprocess_sentence(sentence)

  inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  #hidden = [tf.zeros((1, units))]
  hidden = [tf.zeros((1, units)) for i in range(2)]

  enc_hidden = encoder(inputs, hidden)

  #dec_hidden = enc_hidden
  dec_hidden = enc_hidden[1]

  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden = decoder(dec_input, dec_hidden)

    # storing the attention weights to plot later on
    predicted_id = tf.argmax(predictions[0]).numpy()
    result += targ_lang.index_word[predicted_id] + ' '

    if targ_lang.index_word[predicted_id] == '<end>':
      return result, sentence

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence

In [49]:
def translate(sentence):
  result, sentence = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

## Restore the latest checkpoint and test

In [50]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [51]:
translate('Здесь хорошо.')

Input: <start> здесь хорошо . <end>
Predicted translation: it's ok here . <end> 


In [52]:
translate('Я не смогу поехать.')

Input: <start> я не смогу поехать . <end>
Predicted translation: i can't go . <end> 


In [53]:
translate(u'Вы еще дома?')

Input: <start> вы еще дома ? <end>
Predicted translation: are you still at home ? <end> 


In [54]:
translate(u'Вы все еще дома?')

Input: <start> вы все еще дома ? <end>
Predicted translation: are you still at home ? <end> 


In [55]:
translate(u'Попробуй сделать это.')

Input: <start> попробуй сделать это . <end>
Predicted translation: try to do that . <end> 


In [56]:
translate(u'Я люблю, когда идет снег.')

Input: <start> я люблю , когда идет снег . <end>
Predicted translation: i like to try it . <end> 


In [57]:
translate(u'Я никогда такого не делаю.')

Input: <start> я никогда такого не делаю . <end>
Predicted translation: i never do that . <end> 


In [58]:
translate(u'За плечами у меня не было ни опыта работы')

Input: <start> за плечами у меня не было ни опыта работы <end>
Predicted translation: i ran about tom's . <end> 


In [60]:
translate(u'За плечами у меня не было ни опыта работы, ни подготовки.')

KeyError: ignored

In [61]:
translate(u'Оглядываясь по сторонам, я видел вокруг себя целеустремленных молодых людей')

KeyError: ignored

In [62]:
translate(u'Оглядываясь по сторонам, я видел вокруг себя целеустремленных молодых людей, у которых уже имелись начальные ученые степени в сфере бизнеса')

KeyError: ignored

In [63]:
translate(u'За плечами у них уже был опыт аналитической работы в самых престижных фирмах Уолл-стрит. Конечно же, я чувствовал себя не в своей тарелке.')

KeyError: ignored

# Вывод:
при замене слоя GRU на BidirectionalGRU судя по лоссу на 2 эпохах качество повысилось. При добавлении еще одного слоя BidirectionalGRU  наоборот ухудшилось. Оставил закомменченным код для энкодера с  GRU. Править пришлось почти все последующие функции. Для сложных предложений выбранного словаря недостаточно.